In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Set the working directory
cbis_path = '/content/drive/MyDrive/colab_scripts/repository/datasets'
os.chdir(cbis_path)

# Confirm current directory
print("Current Directory:", os.getcwd())

Current Directory: /content/drive/MyDrive/colab_scripts/repository/datasets


In [ ]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

In [ ]:
# Load the CSV file
df = pd.read_csv('CBIS-DDSM/csv/mass_train_jpg2.csv')

def get_yolo_bbox(mask_path, show=True):
    """Extract YOLO-format bbox and optionally display it."""
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"Warning: Could not read mask: {mask_path}")
        return None

    # Find non-zero regions
    contours, _ = cv2.findContours((mask > 0).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print(f"No contours found in mask: {mask_path}")
        return None

    # Get bounding rect
    x, y, w, h = cv2.boundingRect(contours[0])
    img_h, img_w = mask.shape

    # Normalize to [0, 1]
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    w_norm = w / img_w
    h_norm = h / img_h

    bbox = (x_center, y_center, w_norm, h_norm)

    if show:
        # Show image with bounding box
        img_vis = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
        cv2.rectangle(img_vis, (x, y), (x + w, y + h), (255, 0, 0), 2)

        plt.figure(figsize=(5, 5))
        plt.imshow(img_vis)
        plt.title(f"YOLO BBox: {bbox}")
        plt.axis('off')
        plt.show()

    return bbox

# Process each row with visualization
yolo_bboxes = []
for idx, row in df.iterrows():
    print(f"Processing row {idx + 1}/{len(df)}")
    bbox = get_yolo_bbox(row['jpg ROI mask file path'], show=True)
    yolo_bboxes.append(bbox)

# Add bboxes to DataFrame
df['yolo_bbox'] = yolo_bboxes

# Save the new DataFrame to a CSV file
df.to_csv('CBIS-DDSM/csv/mass_train_jpg2_bbox.csv', index=False)


